<a href="https://colab.research.google.com/github/nimaone/adaptive_teacher/blob/main/MIC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Fri Jan 27 14:12:34 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P0    28W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version

1.13.1+cu116 True
gcc (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0
Copyright (C) 2019 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [ ]:
!pip install torch==1.12.0+cu113 torchvision==0.13.0+cu113 torchaudio==0.12.0 --extra-index-url https://download.pytorch.org/whl/cu113


In [1]:
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version

1.12.0+cu113 True
gcc (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0
Copyright (C) 2019 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [ ]:
!pip install ninja==1.10.2.3 yacs==0.1.8 cython==0.29.28 matplotlib==3.5.1 tqdm==4.63.0 opencv-python==4.5.5.64


In [ ]:
!pip install timm==0.6.11 kornia==0.5.8 einops==0.4.1


In [4]:
import os
HOME = os.getcwd()
print(HOME)

/content


In [ ]:
%cd {HOME}
!git clone https://github.com/cocodataset/cocoapi.git
%cd cocoapi/PythonAPI
!python setup.py build_ext install

In [ ]:
%cd {HOME}
!git clone https://github.com/mcordts/cityscapesScripts.git
%cd cityscapesScripts/
!python setup.py build_ext install

In [ ]:
%cd {HOME}
!git clone https://github.com/lhoyer/MIC
%cd MIC/det
# the following will install the lib with
# symbolic links, so that you can modify
# the files if you want and won't need to
# re-build it
!python setup.py build develop

In [8]:
!ls

build		 INSTALL.md	     maskrcnn_benchmark.egg-info  tests
configs		 LICENSE	     README.md			  tools
environment.yml  maskrcnn_benchmark  setup.py


In [ ]:
%cd {HOME}

# copy required file to cityscapesScripts toolkit
!cp tool/cityscapes/instances2dict_with_polygons.py $INSTALL_DIR/cityscapesScripts/evaluation

# recompile cityscapesScripts
%cd cityscapesScripts/
!python setup.py build_ext install

!pip install h5py==3.6.0 scipy==1.8.0

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !wget https://deepblue.lib.umich.edu/data/downloads/ks65hc58r

In [ ]:
# !wget http://droplab-files.engin.umich.edu/repro_10k_annotations.tgz

In [ ]:
# !wget http://droplab-files.engin.umich.edu/repro_image_sets.tgz

In [ ]:
# !gdown 1-1Iow33KqdaNQcrK4v9Zk568IZVYZ1UV
# !gdown 1-0WQV7BerZb2ieF0LYzoFlO9NFjy4gy7
# !gdown 1-1rhR_4ynP162kL1UrmWQex8tyj6Ki3F


In [5]:
!mkdir /content/datasets

In [ ]:
!tar -zxvf /content/repro_10k_images.tgz -C /content/datasets
!tar -zxvf /content/repro_10k_annotations.tgz -C /content/datasets
!tar -zxvf /content/repro_image_sets.tgz -C /content/datasets

In [21]:
!mv /content/datasets/VOC2012 /content/datasets/Sim10k


In [ ]:
!wget --keep-session-cookies --save-cookies=cookies.txt --post-data 'username=kiaei&password=UtP8!pbXp8FsQs2&submit=Login' https://www.cityscapes-dataset.com/login/

In [ ]:
!wget --load-cookies cookies.txt --content-disposition https://www.cityscapes-dataset.com/file-handling/?packageID=3

In [ ]:
!wget --load-cookies cookies.txt --content-disposition https://www.cityscapes-dataset.com/file-handling/?packageID=1

In [12]:
!unzip -q /content/leftImg8bit_trainvaltest.zip -d /content/datasets

In [13]:
!unzip -q /content/gtFine_trainvaltest.zip -d /content/datasets/cityscapes

In [14]:
!mv /content/datasets/leftImg8bit /content/datasets/cityscapes

In [ ]:
!rm -r  /content/leftImg8bit_trainvaltest.zip

In [ ]:
!pip install fiftyone

In [ ]:
import fiftyone as fo
from fiftyone import ViewField as F


In [ ]:

imgs_path = "/content/datasets/Sim10k/JPEGImages"
labels_path = "/content/datasets/Sim10k/Annotations"


# name = 'visdrone'
# The dataset or view to export
dataset = fo.Dataset.from_dir(
    dataset_type=fo.types.VOCDetectionDataset,
    data_path=imgs_path,
    labels_path=labels_path,
)


In [ ]:
# session = fo.launch_app(dataset)


In [ ]:
car_view = dataset.filter_labels(
    "ground_truth", (F("label") == "car") 
)

In [ ]:
session = fo.launch_app(view=car_view)


In [ ]:
# export_dir = "/content/datasets/Sim10k"
labels_path  = "/content/datasets/Sim10k/train_only_car.json"

# The name of the sample field containing the label that you wish to export
# Used when exporting labeled datasets (e.g., classification or detection)
label_field = "ground_truth"
dataset_type = fo.types.COCODetectionDataset  # for example

car_view.export(
    labels_path=labels_path,
    dataset_type=dataset_type,
    label_field=label_field,
    export_media=False
)

In [ ]:
%pip install cityscapesscripts

In [18]:
%cd {HOME}

/content


In [21]:
%cd /content/MIC/det/tools/cityscapes

/content/MIC/det/tools/cityscapes


In [ ]:
!python convert_cityscapes_to_caronly_coco.py --dataset cityscapes_car_only\
  --outdir '/content/datasets/cityscapes'\
  --datadir '/content/datasets/cityscapes/'

In [ ]:
!python convert_sim10k_to_coco.py --dataset sim10k\
  --outdir '/content/datasets/Sim10k/'\
  --datadir '/content/datasets/Sim10k/Annotations'

In [29]:
!mv  /content/datasets/cityscapes/leftImg8bit/train/*/*.png /content/datasets/cityscapes/leftImg8bit/train/
!mv  /content/datasets/cityscapes/leftImg8bit/val/*/*.png /content/datasets/cityscapes/leftImg8bit/val/
!mv  /content/datasets/cityscapes/leftImg8bit/test/*/*.png /content/datasets/cityscapes/leftImg8bit/test/


In [3]:
%cd /content/MIC/det

[Errno 2] No such file or directory: '{HOME}'
/content/MIC/det
/content/MIC/det


In [4]:
!python tools/train_net.py --config-file configs/da_faster_rcnn/e2e_da_faster_rcnn_R_50_FPN_masking_cs.yaml


2023-01-27 15:39:39,421 maskrcnn_benchmark INFO: Using 1 GPUs
2023-01-27 15:39:39,421 maskrcnn_benchmark INFO: Namespace(config_file='configs/da_faster_rcnn/e2e_da_faster_rcnn_R_50_FPN_masking_cs.yaml', deterministic=False, distributed=False, local_rank=0, opts=[], seed=1, skip_test=False)
2023-01-27 15:39:39,421 maskrcnn_benchmark INFO: Collecting env info (might take some time)
2023-01-27 15:39:40,701 maskrcnn_benchmark INFO: 
PyTorch version: 1.12.0+cu113
Is debug build: False
CUDA used to build PyTorch: 11.3
ROCM used to build PyTorch: N/A

OS: Ubuntu 20.04.5 LTS (x86_64)
GCC version: (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0
Clang version: 10.0.0-4ubuntu1 
CMake version: version 3.22.6
Libc version: glibc-2.31

Python version: 3.8.10 (default, Nov 14 2022, 12:59:47)  [GCC 9.4.0] (64-bit runtime)
Python platform: Linux-5.10.147+-x86_64-with-glibc2.29
Is CUDA available: True
CUDA runtime version: 11.2.152
GPU models and configuration: GPU 0: Tesla T4
Nvidia driver version: 510.47.03
cuD